In [78]:
import pandas as pd
import numpy as np
import tensorflow as tf 
from tensorflow import keras
import pickle
from sklearn.metrics import mean_squared_error
import seaborn as sns 
import matplotlib.pyplot as plt
from numpy.lib.stride_tricks import sliding_window_view
import datetime as dt
import random
import os 
import h5py

BATCH_SIZE = 32
PATIENCE = 20
MAX_EPOCH = 200 



In [80]:
test_data = np.load('tes_data_sli.npy')
test_lab = np.load('tes_lab.npy')

train_data = np.load('tra_data_sli.npy')
train_lab = np.load('tra_lab.npy')

val_data = np.load('val_data_sli.npy')
val_lab = np.load('val_lab.npy')



input_size = train_data.shape[1]
output_size = train_lab.shape[1]


In [81]:
print("Number of training samples are {}".format(train_data.shape[0]))
print("Number of validation samples are {}".format(val_data.shape[0]))
print("Number of test samples are {}".format(test_data.shape[0]))

Number of training samples are 725
Number of validation samples are 204
Number of test samples are 100


In [82]:
train_data.shape

(725, 280, 1)

# LSTM Model Code

In [83]:
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(32,activation='sigmoid',return_sequences=True,input_shape=(train_data.shape[1],train_data.shape[2])),
    tf.keras.layers.LSTM(32,activation='sigmoid',return_sequences=False),
    tf.keras.layers.Dense(units=56)
]
)
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_16 (LSTM)              (None, 280, 32)           4352      
                                                                 
 lstm_17 (LSTM)              (None, 32)                8320      
                                                                 
 dense_8 (Dense)             (None, 56)                1848      
                                                                 
Total params: 14,520
Trainable params: 14,520
Non-trainable params: 0
_________________________________________________________________


In [84]:

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch : ((1e-4)*(10**(epoch/40))) if (1e-4)*(10**(epoch/20)) < 1e-2 else 10**-2
)

adam = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)

ear_stop = tf.keras.callbacks.EarlyStopping(
            monitor="val_loss",mode='min',min_delta=5e-4,patience=20,verbose=1
)

model_chk = tf.keras.callbacks.ModelCheckpoint(
            'best_model.h5',monitor="val_loss",mode='min',verbose=1,save_best_only=True
)


In [85]:
model.compile(optimizer=adam,loss=tf.keras.losses.Huber(),metrics=["mean_squared_error"])

In [86]:
history = model.fit(x=train_data,y=train_lab, epochs=MAX_EPOCH, batch_size=BATCH_SIZE,validation_data=(val_data, val_lab), verbose=1,callbacks=[lr_scheduler, model_chk,ear_stop])

Epoch 1/200
23/23 [==============================] - ETA: 0s - loss: 0.3825 - mean_squared_error: 0.8599
Epoch 1: val_loss improved from inf to 0.38015, saving model to best_model.h5
23/23 [==============================] - 8s 200ms/step - loss: 0.3825 - mean_squared_error: 0.8599 - val_loss: 0.3802 - val_mean_squared_error: 0.8413 - lr: 1.0000e-04
Epoch 2/200
23/23 [==============================] - ETA: 0s - loss: 0.3633 - mean_squared_error: 0.8156
Epoch 2: val_loss improved from 0.38015 to 0.36061, saving model to best_model.h5
23/23 [==============================] - 4s 168ms/step - loss: 0.3633 - mean_squared_error: 0.8156 - val_loss: 0.3606 - val_mean_squared_error: 0.7963 - lr: 1.0593e-04
Epoch 3/200
23/23 [==============================] - ETA: 0s - loss: 0.3435 - mean_squared_error: 0.7704
Epoch 3: val_loss improved from 0.36061 to 0.34045, saving model to best_model.h5
23/23 [==============================] - 4s 170ms/step - loss: 0.3435 - mean_squared_error: 0.7704 - val_lo

Epoch 48/200
23/23 [==============================] - ETA: 0s - loss: 0.0927 - mean_squared_error: 0.2102
Epoch 48: val_loss improved from 0.10842 to 0.10779, saving model to best_model.h5
23/23 [==============================] - 4s 169ms/step - loss: 0.0927 - mean_squared_error: 0.2102 - val_loss: 0.1078 - val_mean_squared_error: 0.2454 - lr: 0.0100
Epoch 49/200
23/23 [==============================] - ETA: 0s - loss: 0.0931 - mean_squared_error: 0.2123
Epoch 49: val_loss did not improve from 0.10779
23/23 [==============================] - 4s 160ms/step - loss: 0.0931 - mean_squared_error: 0.2123 - val_loss: 0.1130 - val_mean_squared_error: 0.2519 - lr: 0.0100
Epoch 50/200
23/23 [==============================] - ETA: 0s - loss: 0.0927 - mean_squared_error: 0.2104
Epoch 50: val_loss improved from 0.10779 to 0.10680, saving model to best_model.h5
23/23 [==============================] - 4s 170ms/step - loss: 0.0927 - mean_squared_error: 0.2104 - val_loss: 0.1068 - val_mean_squared_err

In [87]:
model.evaluate(x=test_data,y=test_lab)

4/4 [==============================] - 0s 44ms/step - loss: 0.0804 - mean_squared_error: 0.1835


[0.08036354184150696, 0.1835242509841919]